In [1]:
import pandas as pd
import sklearn
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

C:\Users\gbm7594\AppData\Local\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
pd.__version__

'2.2.3'

In [3]:
sklearn.__version__

'1.6.1'

In [4]:
#!pip install pyarrow

# Dataset

### Download Yellow Taxi Trip Records - Jan 2023

In [5]:
df_jan = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

### Download Yellow Taxi Trip Records - Jan 2023

In [6]:
df_feb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

# EDA

In [7]:
df_jan.shape

(3066766, 19)

In [8]:
df_jan.head(5)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [9]:
df_jan.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [10]:
df_jan.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
dtype: object

### Trip Duration -  standard deviation

In [11]:
df_jan['rideDuration'] = df_jan.apply(lambda x: (x['tpep_dropoff_datetime'] - x['tpep_pickup_datetime']).total_seconds() / 60, axis=1)

In [12]:
df_jan['rideDuration'].std()

42.59435124195458

### Trip Duration - remove outliers

Keep only the records where the duration was between 1 and 60 minutes (inclusive).

In [13]:
df_jan_filtered = df_jan[df_jan['rideDuration'].between(1, 60, inclusive='both')]

In [14]:
retained_perce = df_jan_filtered.shape[0]*100 /df_jan.shape[0]

In [15]:
retained_perce

98.1220282212598

# One hot encoding - X

Apply one-hot encoding to the pickup and dropoff location IDs

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)

Fit a dictionary vectorizer

Get a feature matrix from it

In [16]:
df_jan_filtered['PULocationID']

0          161
1           43
2           48
3          138
4          107
          ... 
3066761    107
3066762    112
3066763    114
3066764    230
3066765    262
Name: PULocationID, Length: 3009173, dtype: int64

In [17]:
df_jan_filtered['DOLocationID']

0          141
1          237
2          238
3            7
4           79
          ... 
3066761     48
3066762     75
3066763    239
3066764     79
3066765    143
Name: DOLocationID, Length: 3009173, dtype: int64

In [42]:
categorical = ['PULocationID', 'DOLocationID']

In [43]:
X_df= df_jan_filtered[categorical].astype(str)

In [44]:
X_dicts = X_df[categorical].to_dict(orient='records')

In [45]:
dv = DictVectorizer()

In [46]:
X = dv.fit_transform(X_dicts)

In [47]:
X.dtype

dtype('float64')

In [48]:
X.shape

(3009173, 515)

In [49]:
len(X[0].toarray()[0])

515

# Train a regression model

Train a plain linear regression model with default parameters, where duration is the response variable.

Calculate the RMSE of the model on the training data

In [50]:
y = df_jan_filtered['rideDuration']

In [51]:
lr = LinearRegression()

In [52]:
lr.fit(X, y)

LinearRegression()

In [53]:
y_pred = lr.predict(X)

In [54]:
rmse = np.sqrt(mean_squared_error(y, y_pred))

In [55]:
rmse

7.649262236295703

# Evaluate the model

In [56]:
df_feb['rideDuration'] = df_feb.apply(lambda x: (x['tpep_dropoff_datetime'] - x['tpep_pickup_datetime']).total_seconds() / 60, axis=1)

In [57]:
df_feb_filtered = df_feb[df_feb['rideDuration'].between(1, 60, inclusive='both')]

In [58]:
X_test_df= df_feb_filtered[categorical].astype(str)

In [59]:
X_test_dicts = X_test_df[categorical].to_dict(orient='records')

In [60]:
X_test = dv.transform(X_test_dicts)

In [61]:
y_test = df_feb_filtered['rideDuration']

In [62]:
y_hat = lr.predict(X_test)

In [63]:
rmse = np.sqrt(mean_squared_error(y_test, y_hat))

In [64]:
rmse

7.811812822882009